In [95]:
pip install catboost

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# roc curve and auc score
from sklearn.datasets import make_classification
# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

from sklearn.preprocessing import LabelEncoder
# Import Dependencies
%matplotlib inline

# Start Python Imports
import math, time, random, datetime

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split



plt.style.use('seaborn-whitegrid')

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

# Machine learning
import catboost
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier, Pool, cv

# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
df=pd.read_csv('/content/train_fNxu4vz.csv')

In [0]:
df1=pd.read_csv('/content/test_fjtUOL8.csv')

In [4]:
df.shape

(164309, 14)

In [5]:
df1.shape

(109541, 13)

In [6]:
df.isnull().sum()

Loan_ID                        0
Loan_Amount_Requested          0
Length_Employed             7371
Home_Owner                 25349
Annual_Income              25102
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    88379
Number_Open_Accounts           0
Total_Accounts                 0
Gender                         0
Interest_Rate                  0
dtype: int64

In [7]:
df.head()

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
1,10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3
2,10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3
3,10000004,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3
4,10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1


In [8]:
## Let us capture all the nan values
## First lets handle Categorical features which are missing
features_nan=[feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtypes=='O']

for feature in features_nan:
    print("{}: {}% missing values".format(feature,np.round(df[feature].isnull().mean(),4)))

Length_Employed: 0.0449% missing values
Home_Owner: 0.1543% missing values


In [9]:
## Let us capture all the nan values
## First lets handle Categorical features which are missing
features_nan=[feature for feature in df1.columns if df1[feature].isnull().sum()>1 and df1[feature].dtypes=='O']

for feature in features_nan:
    print("{}: {}% missing values".format(feature,np.round(df1[feature].isnull().mean(),4)))

Length_Employed: 0.0451% missing values
Home_Owner: 0.1526% missing values


In [10]:
def replace_cat_feature(dataset,features_nan):
    data=df.copy()
    data[features_nan]=data[features_nan].fillna('Missing')
    return data

df=replace_cat_feature(df,features_nan)

df[features_nan].isnull().sum()

Length_Employed    0
Home_Owner         0
dtype: int64

In [11]:
def replace_cat_feature(dataset,features_nan):
    data1=df1.copy()
    data1[features_nan]=data1[features_nan].fillna('Missing')
    return data1

df1=replace_cat_feature(df1,features_nan)

df1[features_nan].isnull().sum()

Length_Employed    0
Home_Owner         0
dtype: int64

In [12]:
## Now lets check for numerical variables the contains missing values
numerical_with_nan=[feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtypes!='O']

## We will print the numerical nan variables and percentage of missing values

for feature in numerical_with_nan:
    print("{}: {}% missing value".format(feature,np.around(df[feature].isnull().mean(),4)))

Annual_Income: 0.1528% missing value
Months_Since_Deliquency: 0.5379% missing value


In [13]:
## Now lets check for numerical variables the contains missing values
numerical_with_nan=[feature for feature in df1.columns if df1[feature].isnull().sum()>1 and df1[feature].dtypes!='O']

## We will print the numerical nan variables and percentage of missing values

for feature in numerical_with_nan:
    print("{}: {}% missing value".format(feature,np.around(df1[feature].isnull().mean(),4)))

Annual_Income: 0.1543% missing value
Months_Since_Deliquency: 0.5373% missing value


In [14]:

## Replacing the numerical Missing Values

for feature in numerical_with_nan:
    ## We will replace by using median since there are outliers
    median_value=df[feature].median()
    
    ## create a new feature to capture nan values
    df[feature+'nan']=np.where(df[feature].isnull(),1,0)
    df[feature].fillna(median_value,inplace=True)
    
df[numerical_with_nan].isnull().sum()

Annual_Income              0
Months_Since_Deliquency    0
dtype: int64

In [15]:
## Replacing the numerical Missing Values

for feature in numerical_with_nan:
    ## We will replace by using median since there are outliers
    median_value=df1[feature].median()
    
    ## create a new feature to capture nan values
    df1[feature+'nan']=np.where(df1[feature].isnull(),1,0)
    df1[feature].fillna(median_value,inplace=True)
    
df1[numerical_with_nan].isnull().sum()

Annual_Income              0
Months_Since_Deliquency    0
dtype: int64

In [16]:
df.head()

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate,Annual_Incomenan,Months_Since_Deliquencynan
0,10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,31.0,9,14,Female,1,0,1
1,10000002,"30,000",4 years,Mortgage,63000.0,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3,1,0
2,10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,31.0,12,16,Male,3,0,1
3,10000004,"16,000",< 1 year,Missing,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,31.0,16,22,Male,3,0,1
4,10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,31.0,19,30,Female,1,0,1


In [0]:
#df['Months_Since_Deliquency']=df['Months_Since_Deliquency'].fillna(0)
#df['Annual_Income']=df['Annual_Income'].fillna(0)
#df['Home_Owner']=df['Home_Owner'].fillna('aaa')
#df['Length_Employed']=df['Length_Employed'].fillna('x')
#df1['Months_Since_Deliquency']=df1['Months_Since_Deliquency'].fillna(0)
#df1['Annual_Income']=df1['Annual_Income'].fillna(0)
#df1['Home_Owner']=df1['Home_Owner'].fillna('aaa')
#df1['Length_Employed']=df1['Length_Employed'].fillna('x')

In [31]:
df.isnull().sum()

Loan_ID                       0
Loan_Amount_Requested         0
Length_Employed               0
Home_Owner                    0
Annual_Income                 0
Income_Verified               0
Purpose_Of_Loan               0
Debt_To_Income                0
Inquiries_Last_6Mo            0
Months_Since_Deliquency       0
Number_Open_Accounts          0
Total_Accounts                0
Gender                        0
Interest_Rate                 0
Annual_Incomenan              0
Months_Since_Deliquencynan    0
dtype: int64

In [0]:
#le = LabelEncoder()
#df['Length_Employed'] = le.fit_transform(df['Length_Employed']) 
#df['Home_Owner'] = le.fit_transform(df['Home_Owner'])
#df['Income_Verified'] = le.fit_transform(df['Income_Verified'])
#df['Purpose_Of_Loan'] = le.fit_transform(df['Purpose_Of_Loan'])
#df['Gender'] = le.fit_transform(df['Gender'])
#df['Loan_Amount_Requested'] = le.fit_transform(df['Loan_Amount_Requested'])

In [0]:
#df1['Length_Employed'] = le.fit_transform(df1['Length_Employed']) 
#df1['Home_Owner'] = le.fit_transform(df1['Home_Owner'])
#df1['Income_Verified'] = le.fit_transform(df1['Income_Verified'])
#df1['Purpose_Of_Loan'] = le.fit_transform(df1['Purpose_Of_Loan'])
#df1['Gender'] = le.fit_transform(df1['Gender'])
#df1['Loan_Amount_Requested'] = le.fit_transform(df1['Loan_Amount_Requested'])

In [0]:
X = df[[col for col in df.columns if col not in ('Interest_Rate')]].values  #independent variables 
y = df['Interest_Rate'].values  #dependent variable 

In [0]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [0]:

train_pool = Pool(X_train, 
                  y_train,
                  cat_features=[1,2,3,5,6,12])

In [0]:
catboost_model = CatBoostClassifier(iterations=160,
                                    random_state=42,
                                    max_depth=4,
                                   bagging_temperature=0.7,
                                    l2_leaf_reg=8,
                                    grow_policy='Depthwise',
                                    learning_rate=0.3,task_type='GPU',cat_features=[1,2,3,5,6,12])



In [127]:
catboost_model.fit(X_train,y_train)

0:	learn: 1.0447274	total: 9.88ms	remaining: 1.57s
1:	learn: 1.0146167	total: 18.7ms	remaining: 1.48s
2:	learn: 0.9958308	total: 26.9ms	remaining: 1.41s
3:	learn: 0.9814766	total: 35ms	remaining: 1.37s
4:	learn: 0.9714125	total: 43.4ms	remaining: 1.34s
5:	learn: 0.9635327	total: 51.7ms	remaining: 1.33s
6:	learn: 0.9574124	total: 59.9ms	remaining: 1.31s
7:	learn: 0.9526018	total: 68.1ms	remaining: 1.29s
8:	learn: 0.9487035	total: 76.4ms	remaining: 1.28s
9:	learn: 0.9446252	total: 84.8ms	remaining: 1.27s
10:	learn: 0.9412845	total: 93.2ms	remaining: 1.26s
11:	learn: 0.9386352	total: 101ms	remaining: 1.25s
12:	learn: 0.9361004	total: 109ms	remaining: 1.24s
13:	learn: 0.9341812	total: 118ms	remaining: 1.23s
14:	learn: 0.9324214	total: 126ms	remaining: 1.22s
15:	learn: 0.9307844	total: 145ms	remaining: 1.3s
16:	learn: 0.9295614	total: 152ms	remaining: 1.28s
17:	learn: 0.9283683	total: 160ms	remaining: 1.26s
18:	learn: 0.9273291	total: 167ms	remaining: 1.24s
19:	learn: 0.9260893	total: 175ms

In [0]:
y_Pred = catboost_model.predict(X_test)

In [129]:
from sklearn.metrics import f1_score

f1_score(y_test, y_Pred,average='weighted') 


0.5357956544969503

In [0]:
predictions = catboost_model.predict(df1.values)

In [131]:
submission = pd.DataFrame()
submission['Loan_ID'] = df1['Loan_ID']
submission['Interest_Rate'] = predictions # our model predictions on the test dataset
submission.head(50)

,Loan_ID,Interest_Rate
0,10164310,2
1,10164311,1
2,10164312,3
3,10164313,2
4,10164314,2
5,10164315,3
6,10164316,3
7,10164317,3
8,10164318,3
9,10164319,3


In [0]:
submission.to_csv("csv14.csv",index=False)